In [2]:
import pandas as pd

In [31]:
hce_bids = pd.read_csv('data/cash_flow_hce_bid.csv', index_col=0, parse_dates=True)

In [32]:
transformer_interval = pd.read_csv('data/cash_flow_transformer_interval.csv', index_col=0, parse_dates=True)

In [33]:
meter_interval = pd.read_csv('data/cash_flow_meter_interval.csv', index_col=0, parse_dates=True)

In [34]:
market_interval = pd.read_csv('data/cash_flow_market_interval.csv', index_col=0, parse_dates=True).iloc[:1]

In [35]:
start_times = list(market_interval["start_time"])
start_times

['2021-07-22T11:42:00']

In [36]:
p_clear = list(market_interval["p_clear"])
p_clear

[0.11]

### Sum all ((qmtp in TABLE meter_intervals for which p_bid <= p_clear))*p_clear

In [21]:
# meter_interval.loc[(meter_ineterval.p_bid <= )]
merge_meter_market_interval = meter_interval.merge(market_interval, on="start_time", how="left")

In [39]:
x = merge_meter_market_interval.loc[merge_meter_market_interval.p_bid <= merge_meter_market_interval.p_clear]

In [28]:
x["payments_to_pv_owner"] = x["p_clear"]*x["q_bid"]

/Users/derins/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
payments_to_pv_owners = x.groupby("start_time")["payments_to_pv_owner"].sum()
payments_to_pv_owners

start_time
2021-07-22T11:42:00    1.456876
Name: payments_to_pv_owner, dtype: float64

###  (q from TABLE transformer_intervals at start_time == t) * (p_bid in TABLE hce_bids at start_time == t)
### Use p_bid for which is_supply = 1 if q >= 0
### Use p_bid for which is_supply = 0 if q < 0


In [40]:
merge_transformer_market_interval = transformer_interval.merge(market_interval, on="start_time", how="left")